In [3]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from datetime import timedelta

In [4]:
data = pd.read_csv('trades.csv',sep=',')

In [5]:
data.head()

,MS US Equity,Unnamed: 1,Unnamed: 2,TXN US Equity,Unnamed: 4
0,Dates,Close,NaN,Dates,Close
1,23/1/2019 9:30,42.57,NaN,23/1/2019 9:30,96.63
2,23/1/2019 9:31,42.46,NaN,23/1/2019 9:31,96.42
3,23/1/2019 9:32,42.49,NaN,23/1/2019 9:32,96.51
4,23/1/2019 9:33,42.43,NaN,23/1/2019 9:33,96.81


In [6]:
formatDateTime = '%d/%m/%Y %H:%M'

formatDate = '%Y%m%d'

In [25]:
#Settings

equity = 'TXN'

startTime = '9:20'
endTime = '16:00'
startDate = '7/6/2019'
endDate = '21/7/2019'

minutesDelta = 120

In [125]:
class dayStatistics:
    
    openPrice = 0.0
    closePrice = 0.0
    highPrice = 0.0
    lowPrice = 0.0
    RSI = 0.0
    MACD = 0.0
    dayString = ''
    
    def _init_(self, dateTime):
        self.dayString = dateTime.strftime(formatDate)
        
    def printStats(self):
        print("Open, Close, High, Low, RSI, MACD")
        print(str(self.openPrice)+", "+str(self.closePrice)+", "+str(self.highPrice)+", "+str(self.lowPrice)+", "+str(self.RSI)+", "+str(self.MACD))

In [106]:
def securityIndex(equity, data):    
    ticker = equity + ' US Equity'
    index=0
    counter=0

    for col in data.columns:
        if col==ticker: 
            index = counter
        counter = counter+1
        
    return index    

In [107]:
index = securityIndex(equity, data)
print(index)

3


In [108]:
def processTimeSeries(startTime, startDate, endTime, endDate, equity, data, minutesDelta):

    array2D = []
    array2D.append([])
    array2D.append([])
 
    startDateTime = datetime.strptime(startDate+' '+startTime, formatDateTime)
    endDateTime = datetime.strptime(endDate+' '+endTime, formatDateTime)

    dateTime = startDateTime
    counter = 1
    index = securityIndex(equity, data)
    tradingDay = True
    incrementCounter = True

    while dateTime<=endDateTime:
        if incrementCounter or (recordDateTime<dateTime):
            counter = counter + 1
        if data.iloc[counter,index] != '' and data.iloc[counter,index] != 'nan':
            
            recordDateTime = datetime.strptime(data.iloc[counter,index], formatDateTime)
            #print(recordDateTime)
            if recordDateTime>=dateTime:
                minDelta = (recordDateTime-dateTime).days*24*60
                if recordDateTime == dateTime or (tradingDay==False and minDelta <=30.0):
                    array2D[0].append(dateTime)
                    dateTime = dateTime + timedelta(minutes=minutesDelta)
                    tradingDay = True 
                    incrementCounter = True
                    array2D[1].append(float(data.iloc[counter, index+1]))
                elif recordDateTime>dateTime:
                    eodDateTime = datetime.strptime(str(dateTime.day) +"/"+ str(dateTime.month) + "/"+ str(dateTime.year) +' '+endTime, formatDateTime)
                    if recordDateTime >= eodDateTime:
                        if tradingDay and (eodDateTime.day == recordDateTime.day):
                            array2D[0].append(eodDateTime)
                            array2D[1].append(float(data.iloc[counter, index+1]))
                            #print("Match2")
                        else:
                            incrementCounter = False
                        dateTime = dateTime + timedelta(days=1)
                        dateTime = datetime.strptime(str(dateTime.day) +"/"+ str(dateTime.month) + "/"+ str(dateTime.year) +' '+startTime, formatDateTime)
                    tradingDay = False
                    #print(dateTime)
                    #print(tradingDay)
                    #print((recordDateTime-dateTime).days*24*60)
                
    return array2D

In [109]:
sampleTimeSeries = processTimeSeries(startTime, startDate, endTime, endDate, equity, data, minutesDelta)

In [110]:
#Get EODTimeSeries

def getEODTimeSeries(startDate, endDate):
    startTime = '16:00'
    endTime = '16:00'
    #startDate = '1/2/2019'
    #endDate = '10/3/2019'

    minutesDelta = 1440

    eodTimeSeries = processTimeSeries(startTime, startDate, endTime, endDate, equity, data, minutesDelta)
    return eodTimeSeries

In [185]:
def RSICalc(eodTimeSeries, period=14):
    
    array2D = []
    array2D.append([])
    array2D.append([])
    
    for i in range(1, len(eodTimeSeries[0])):
        array2D[0].append(eodTimeSeries[0][i])
    
    delta = pd.DataFrame(eodTimeSeries[1]).diff()
    
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    # Calculate the SMA
    roll_up2 = up.rolling(window=period).mean()
    roll_down2 = (down.rolling(window=period).mean()).abs()
    
    # Calculate the RSI based on SMA
    RS2 = roll_up2 / roll_down2
    RSI2 = 100.0 - (100.0 / (1.0 + RS2))
    RSI2 = RSI2.values
    #Pandas DataFrame
    for i in range(0, (len(eodTimeSeries[0])-1)):
        array2D[1].append(float(RSI2[i]))

    return array2D

In [186]:
def MACDCalc(eodTimeSeries, shortPeriod=12, longPeriod=26):

    array2D = []
    array2D.append([])
    array2D.append([])
    for i in range(1, len(eodTimeSeries[0])):
        array2D[0].append(eodTimeSeries[0][i])
    
    price = pd.DataFrame(eodTimeSeries[1])
    exp1 = price.ewm(span=shortPeriod, adjust=False).mean()
    exp2 = price.ewm(span=longPeriod, adjust=False).mean()
    macd = exp1-exp2
    macd = macd.values
    for i in range(0, (len(eodTimeSeries[0])-1)):
        array2D[1].append(float(macd[i]))
    
    return array2D

In [187]:
eodTimeSeries = getEODTimeSeries(startDate, endDate)
MACDCalc(eodTimeSeries)

[[datetime.datetime(2019, 6, 9, 16, 0),
  datetime.datetime(2019, 6, 10, 16, 0),
  datetime.datetime(2019, 6, 11, 16, 0),
  datetime.datetime(2019, 6, 12, 16, 0),
  datetime.datetime(2019, 6, 13, 16, 0),
  datetime.datetime(2019, 6, 14, 16, 0),
  datetime.datetime(2019, 6, 16, 16, 0),
  datetime.datetime(2019, 6, 17, 16, 0),
  datetime.datetime(2019, 6, 18, 16, 0),
  datetime.datetime(2019, 6, 19, 16, 0),
  datetime.datetime(2019, 6, 20, 16, 0),
  datetime.datetime(2019, 6, 21, 16, 0),
  datetime.datetime(2019, 6, 23, 16, 0),
  datetime.datetime(2019, 6, 24, 16, 0),
  datetime.datetime(2019, 6, 25, 16, 0),
  datetime.datetime(2019, 6, 26, 16, 0),
  datetime.datetime(2019, 6, 27, 16, 0),
  datetime.datetime(2019, 6, 28, 16, 0),
  datetime.datetime(2019, 6, 30, 16, 0),
  datetime.datetime(2019, 7, 1, 16, 0),
  datetime.datetime(2019, 7, 2, 16, 0),
  datetime.datetime(2019, 7, 4, 16, 0),
  datetime.datetime(2019, 7, 5, 16, 0),
  datetime.datetime(2019, 7, 7, 16, 0),
  datetime.datetime(20

In [188]:
#MACD(eodTimeSeries)

def processOHLC(daysStatsDict, processTimeSeries, formatDate, startDate, endDate):
    
    daysUpdatedStatsDict = daysStatsDict
    eodTimeSeries = getEODTimeSeries(startDate, endDate)
    RSI = RSICalc(eodTimeSeries)
    MACD = MACDCalc(eodTimeSeries)
    
    for i in range(0, len(processTimeSeries[0])):
        dateString = processTimeSeries[0][i].strftime(formatDate)
        price = processTimeSeries[1][i]
        
        record = dayStatistics()
        record.dateString = dateString
         #Determine, open, low, high and close
        
        if dateString in daysUpdatedStatsDict:
            record = daysUpdatedStatsDict[dateString]
            record.closePrice = price
            if price > record.highPrice: record.highPrice = price
            if price < record.lowPrice or record.lowPrice == 0.0 : record.lowPrice = price    
        else:
            record.openPrice = price
            
        daysUpdatedStatsDict[dateString] = record
    
    for i in range(0, len(RSI[0])):
        dateString = RSI[0][i].strftime(formatDate)
        if dateString in daysUpdatedStatsDict:
            record = daysUpdatedStatsDict[dateString]
            record.RSI = RSI[1][i]
            record.MACD = MACD[1][i]
            #print(record.MACD)
            #print(dateString)
            daysUpdatedStatsDict[dateString] = record
        
        
    return daysUpdatedStatsDict 

In [189]:
daysStatsDict = {}
daysStats = processOHLC(daysStatsDict, sampleTimeSeries, formatDate, startDate, endDate)
#daysStats.printStats()

In [190]:
record = daysStats['20190705']
record.printStats()
#record.closePrice
#record.lowPrice

Open, Close, High, Low, RSI, MACD
114.905, 115.82, 115.82, 115.17, 69.14619883040936, 1.4157116431365466


In [194]:
import math

def finalDataFrame(daysStatsDict, processTimeSeries, formatDate, startDate, endDate):
    
    daysStats = processOHLC(daysStatsDict, processTimeSeries, formatDate, startDate, endDate)
    df = pd.DataFrame(index=range(len(processTimeSeries[0])),columns=range(1+4+2))
    
    for i in range(0, len(processTimeSeries[1])):
        dateString = processTimeSeries[0][i].strftime(formatDate)
        record = daysStats[dateString]
        
        df.iloc[i, 0] = processTimeSeries[1][i]
        df.iloc[i, 1] = record.openPrice
        df.iloc[i, 2] = record.closePrice
        df.iloc[i, 3] = record.highPrice
        df.iloc[i, 4] = record.lowPrice
        df.iloc[i, 5] = record.RSI
        if math.isnan(record.RSI):
            df.iloc[i, 5] = 0.0
        df.iloc[i, 6] = record.MACD
        
    return df   
        

In [291]:
df = finalDataFrame(daysStatsDict, sampleTimeSeries, formatDate, startDate, endDate)

In [292]:
df

,0,1,2,3,4,5,6
0,110.6,110.6,111.088,111.63,110.6,0,0
1,111.63,110.6,111.088,111.63,110.6,0,0
2,111.27,110.6,111.088,111.63,110.6,0,0
3,111.088,110.6,111.088,111.63,110.6,0,0
4,111.805,111.805,112.93,113.04,111.805,0,0.0506553
5,112.75,111.805,112.93,113.04,111.805,0,0.0506553
6,113.04,111.805,112.93,113.04,111.805,0,0.0506553
7,112.93,111.805,112.93,113.04,111.805,0,0.0506553
8,114.08,114.08,112.85,114.08,112.305,0,0.155577
9,113.63,114.08,112.85,114.08,112.305,0,0.155577


In [293]:

scaler = MinMaxScaler()
Y_test = df.loc[111:118,2]
df.loc[:,0] = MinMaxScaler().fit_transform(df.loc[:, 0].values.reshape(-1,1))
df.loc[:,1] = MinMaxScaler().fit_transform(df.loc[:, 1].values.reshape(-1,1))
df.loc[:,2] = scaler.fit_transform(df.loc[:, 2].values.reshape(-1,1))
df.loc[:,3] = MinMaxScaler().fit_transform(df.loc[:, 3].values.reshape(-1,1))
df.loc[:,4] = MinMaxScaler().fit_transform(df.loc[:, 4].values.reshape(-1,1))
df.loc[:,5] = MinMaxScaler().fit_transform(df.loc[:, 5].values.reshape(-1,1))
df.loc[:,6] = MinMaxScaler().fit_transform(df.loc[:, 6].values.reshape(-1,1))

X_train = df.loc[0:110, df.columns != 2]
X_test =  df.loc[111:118, df.columns != 2]
Y_train = df.loc[0:110, 2]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [294]:
X_train.shape
Y_train.shape

(111,)

In [295]:
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1],1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1],1)

In [301]:
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

model = Sequential()
model.add(LSTM(units = 50,return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 50))
model.add(Dropout(0.2))

model.add(Dense(units = 1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

model.fit(X_train, Y_train, epochs = 200, batch_size = 32)


Epoch 1/200
111/111 [==============================] - 9s 77ms/step - loss: 0.3360
Epoch 2/200
111/111 [==============================] - 0s 1ms/step - loss: 0.2640
Epoch 3/200
111/111 [==============================] - 0s 1ms/step - loss: 0.1565
Epoch 4/200
111/111 [==============================] - 0s 984us/step - loss: 0.0464
Epoch 5/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0769
Epoch 6/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0488
Epoch 7/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0319
Epoch 8/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0456
Epoch 9/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0399
Epoch 10/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0235
Epoch 11/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0266
Epoch 12/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0233
Epoch 13/2

111/111 [==============================] - 0s 870us/step - loss: 0.0066
Epoch 98/200
111/111 [==============================] - 0s 881us/step - loss: 0.0051
Epoch 99/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0059
Epoch 100/200
111/111 [==============================] - 0s 971us/step - loss: 0.0058
Epoch 101/200
111/111 [==============================] - 0s 881us/step - loss: 0.0050
Epoch 102/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0061
Epoch 103/200
111/111 [==============================] - 0s 996us/step - loss: 0.0060
Epoch 104/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0063
Epoch 105/200
111/111 [==============================] - 0s 918us/step - loss: 0.0061
Epoch 106/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0058
Epoch 107/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0058
Epoch 108/200
111/111 [==============================] - 0s 994us/step - loss: 0

Epoch 193/200
111/111 [==============================] - 0s 783us/step - loss: 0.0048
Epoch 194/200
111/111 [==============================] - 0s 984us/step - loss: 0.0052
Epoch 195/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0045
Epoch 196/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0049
Epoch 197/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0053
Epoch 198/200
111/111 [==============================] - 0s 980us/step - loss: 0.0057
Epoch 199/200
111/111 [==============================] - 0s 845us/step - loss: 0.0055
Epoch 200/200
111/111 [==============================] - 0s 1ms/step - loss: 0.0049


In [302]:
predictions = scaler.inverse_transform(model.predict(X_test))
predictions

array([[117.6105  ],
       [117.527176],
       [117.826675],
       [117.97776 ],
       [118.455215],
       [118.10479 ],
       [118.27299 ],
       [117.9618  ]], dtype=float32)

In [303]:
Y_test

111    118.53
112    118.53
113    118.53
114    118.53
115    117.33
116    117.33
117    117.33
118    117.33
Name: 2, dtype: object